In [1]:
from collections import defaultdict
from itertools import zip_longest
import re
import json
import tqdm

In [2]:
FILE_IN_PATH = '../boo/classify/okved_full.txt'
FILE_OUT_PATH = '../boo/classify/okved_full_table.json'

In [3]:
%%time
"""
########################################################################
Скрипт переводит табличку ОКВЭД с разделами и подразделами в JSON 
структуру вида:
{"sector":
    {"subsector":
        {"ok1":
            {"ok2":
                {"ok3":"name"}}}}}
               
По умолчанию ok1, ok2, ok3 переводятся в в строку формата "два численных 
символа" или "впереди стоящий ноль и численный символ" и символ. 
Пример: "01" или "15". 
Можно изменить на численный формат. Пример: 1 или 15

Script makes from ОКВЭД table an output json file structured by following 
scheme:
{"sector":
    {"subsector":
        {"ok1":
            {"ok2":
                {"ok3":"name"}}}}}
By default convert codes to strings and add leading zero in case 
of one number code. Example: "08" или "90".
#########################################################################
"""

dta_dict = defaultdict(lambda: 
                       defaultdict(
                           lambda: defaultdict(
                               lambda: defaultdict(
                                   lambda: defaultdict(
                                       lambda: defaultdict())))))
sectors = []
subsectors = []
codes = []
names = []
with open(FILE_IN_PATH, mode="r+") as file:
    for line in tqdm.tqdm_notebook(file.readlines()):
        if re.match(r"(РАЗДЕЛ)|\t*(РАЗДЕЛ)", line):
            
            sectors.append(re.sub(r"[\n\t]*", "", line))
            sector = re.sub(r"[\n\t]*", "", line)
            
        elif re.match(r"(Подраздел)|\t*(Подраздел)", line):
            
            subsectors.append(re.sub(r"[\n\t]*", "", line))
            subsector = re.sub(r"[\n\t]*", "", line)
        else:
#             aux_line = re.split("[.\t]", line)
#             codes.append(aux_line[0:-1])
#             names.append(aux_line[-1])
            aux_lst = [0, 0, 0]
            aux_line = re.split("(?<=\d)[.\t]", line)
            code_numbers = list(map(int, aux_line[0:-1]))
            code = list(map(sum, list(zip_longest(aux_lst, code_numbers, fillvalue=0))))
            if True:
                dta_dict[sector][subsector][format(code[0], "02")][format(code[1], "02")][format(code[2], "02")] = \
                re.sub(r"\n", "", aux_line[-1])
            else:
                dta_dict[sector][subsector][code[0]][code[1]][code[2]] = re.sub(r"\n", "", aux_line[-1])
            

A Jupyter Widget


Wall time: 48 ms


In [4]:
%%time
dta_dict

Wall time: 0 ns


defaultdict(<function __main__.<lambda>>,
            {'РАЗДЕЛ B РЫБОЛОВСТВО, РЫБОВОДСТВО': defaultdict(<function __main__.<lambda>.<locals>.<lambda>>,
                         {'Подраздел BА РЫБОЛОВСТВО, РЫБОВОДСТВО': defaultdict(<function __main__.<lambda>.<locals>.<lambda>.<locals>.<lambda>>,
                                      {'05': defaultdict(<function __main__.<lambda>.<locals>.<lambda>.<locals>.<lambda>.<locals>.<lambda>>,
                                                   {'00': defaultdict(<function __main__.<lambda>.<locals>.<lambda>.<locals>.<lambda>.<locals>.<lambda>.<locals>.<lambda>>,
                                                                {'00': 'Рыболовство, рыбоводство и предоставление услуг в этих областях'}),
                                                    '01': defaultdict(<function __main__.<lambda>.<locals>.<lambda>.<locals>.<lambda>.<locals>.<lambda>.<locals>.<lambda>>,
                                                                {'00': 'Рыболо

In [5]:
%%time
with open(FILE_OUT_PATH, mode='w+') as file_out:
    file_out.write(json.dumps(dta_dict, ensure_ascii=False))

Wall time: 0 ns
